In [1]:
import requests
import math
import pandas as pd
import json
import os

In [2]:
os.system('mkdir -p cache')

1

In [3]:
import re
from tokenizers import normalizers
from tokenizers.normalizers import BertNormalizer, Sequence, Strip
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Whitespace

normalizer = Sequence([BertNormalizer(clean_text=True,
        handle_chinese_chars=True,
        strip_accents=True,
        lowercase=True), Strip()])
pre_tokenizer = pre_tokenizers.Sequence([Whitespace()])

def normalize(x, min_length = 1):
    if not isinstance(x, str):
        return ''
    normalized = normalizer.normalize_str(x)
    normalized = normalized.replace('\n', ' ')
    normalized = re.sub(' +', ' ', normalized)
    return " ".join([e[0] for e in pre_tokenizer.pre_tokenize_str(normalized) if len(e[0]) > min_length])

### Nettoyer lib entreprises pour query

In [59]:
import urllib.parse

In [60]:
companies = pd.read_csv('liste_entreprises.csv',sep=';', index_col=0)
companies.columns

Index(['name', 'source', 'type'], dtype='object')

In [62]:
list_companies = [] 
for company in companies['name']:
    a = urllib.parse.quote(company)
    list_companies.append(a)

In [16]:
first_year = 2000

In [65]:
assert (affiliation1_type in ['raw_affiliation_string', 'institutions.country_code', 'institutions.ror'])

def get_filename(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, thematic, random_size ):
    filename = f'cache/{normalize(affiliation1_str)}_{normalize(affiliation2_str)}_sample{random_size}_{normalize(thematic)}.json'.replace(' ', '_')
    return filename

In [52]:
BASE_URL = f"https://api.openalex.org/works?filter=publication_year:{first_year}-"

In [53]:
def get_search_field(f):
    if f in ['raw_affiliation_string']:
        return 'raw_affiliation_string.search'
    return f

def get_data(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, thematic ):
    url = BASE_URL + f",{get_search_field(affiliation1_type)}:{affiliation1_str}"
    if affiliation2_str:
        url += f",{get_search_field(affiliation2_type)}:{affiliation2_str}"
    if thematic:
        random_size = None
        url += f"&search={thematic}"
    else:
        random_size = 1000
        url += f"&sample={random_size}&seed=0"
        
    print(url)
    filename = get_filename(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, 
                            thematic, random_size )
    print(filename)
    try:
        data = json.load(open(filename, 'r'))
    except:
        results = []
        res = requests.get(url).json()
        nb_results = res['meta']['count']
        print(nb_results)
        nb_page = math.ceil(nb_results / res['meta']['per_page'])
        results = res['results']
        for p in range(2, nb_page + 1):
            print(p, end=',')
            res = requests.get(url+f"&page={p}").json()
            results += res['results']
        assert(len(results) == nb_results)
        data = {'results': results}
        json.dump(data, open(filename, 'w'))
    return data['results']

In [64]:
all_works = pd.DataFrame()
for company in list_companies:
    data = get_data(company, 'raw_affiliation_string', 'france', 'raw_affiliation_string', thematic)
    df = pd.DataFrame(data)
    all_works = pd.concat([all_works, df])

https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:GENINDEXE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/genindexe_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:AMBIENTIC,raw_affiliation_string.search:france&sample=1000&seed=0
cache/ambientic_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:HYDROCEAN,raw_affiliation_string.search:france&sample=1000&seed=0
cache/hydrocean_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:COVALIA%20INTERACTIVE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/covalia_20interactive_france_sample1000_.json
1
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:Aquitaine%20Science%20Transfert,raw_affiliation_string.search:france&sample=1000&seed=0
cache/a

0
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:TECHNIC%20EXPORT,raw_affiliation_string.search:france&sample=1000&seed=0
cache/technic_20export_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:SCIENCE%20ET%20SURFACE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/science_20et_20surface_france_sample1000_.json
337
2,3,4,5,6,7,8,9,10,11,12,13,14,https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:ALSEAMAR,raw_affiliation_string.search:france&sample=1000&seed=0
cache/alseamar_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:UNAC,raw_affiliation_string.search:france&sample=1000&seed=0
cache/unac_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:AKKA%20INGENIERIE%20PRODUIT,raw_affiliation_string.sea

0
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:ASB%20AEROSPATIALE%20BATTERIES,raw_affiliation_string.search:france&sample=1000&seed=0
cache/asb_20aerospatiale_20batteries_france_sample1000_.json
1
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:IHS%20GLOBAL,raw_affiliation_string.search:france&sample=1000&seed=0
cache/ihs_20global_france_sample1000_.json
1
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:GREENTECH,raw_affiliation_string.search:france&sample=1000&seed=0
cache/greentech_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:ALLICE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/allice_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2016-,raw_affiliation_string.search:AR2I%20SA%20ANALYSES%2C%20RECHERCHES%20ET%20INNOVATION%20INSTRUMENTALE,ra

KeyError: 'meta'

In [66]:
all_works

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,primary_location,host_venue,...,alternate_host_venues,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated
0,https://openalex.org/W3184787821,https://doi.org/10.1080/20961790.2021.1886679,An inter-laboratory study of DNA-based identit...,An inter-laboratory study of DNA-based identit...,0.992930,2021,2021-07-19,{'openalex': 'https://openalex.org/W3184787821...,"{'is_oa': True, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S4210169164', 'is...",...,"[{'id': 'https://openalex.org/S4210169164', 'd...","[https://openalex.org/W83837518, https://opena...","[https://openalex.org/W1525813537, https://ope...",https://api.openalex.org/works/W3184787821/ngrams,"{'The': [0, 34, 67, 181], 'probative': [1], 'v...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2023, 'cited_by_count': 1}]",2023-02-26T14:17:08.281707,2021-08-02,NaN
1,https://openalex.org/W3183842167,https://doi.org/10.1111/faf.12587,The early marine distribution of Atlantic salm...,The early marine distribution of Atlantic salm...,0.890448,2021,2021-07-05,{'openalex': 'https://openalex.org/W3183842167...,"{'is_oa': None, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S103284624', 'iss...",...,"[{'id': 'https://openalex.org/S103284624', 'di...","[https://openalex.org/W186526879, https://open...","[https://openalex.org/W1796212355, https://ope...",https://api.openalex.org/works/W3183842167/ngrams,"{'The': [0, 94, 209], 'survival': [1], 'of': [...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2022, 'cited_by_count': 13}]",2023-02-16T22:46:26.093377,2021-08-02,NaN
2,https://openalex.org/W3017031654,https://doi.org/10.3897/zookeys.926.48679,"Painted black: Iguana melanoderma (Reptilia, S...","Painted black: Iguana melanoderma (Reptilia, S...",0.781384,2020,2020-04-13,{'openalex': 'https://openalex.org/W3017031654...,"{'is_oa': True, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S199213172', 'iss...",...,"[{'id': 'https://openalex.org/S199213172', 'di...","[https://openalex.org/W618738618, https://open...","[https://openalex.org/W2003876219, https://ope...",https://api.openalex.org/works/W3017031654/ngrams,"{'The': [0, 226], 'Lesser': [1, 14, 29], 'Anti...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2023, 'cited_by_count': 1}, {'year':...",2023-02-20T13:08:38.448202,2020-04-24,NaN
3,https://openalex.org/W3164306728,https://doi.org/10.1007/s12686-021-01216-5,A genus-wide analysis of genetic variation to ...,A genus-wide analysis of genetic variation to ...,0.674782,2021,2021-05-22,{'openalex': 'https://openalex.org/W3164306728...,"{'is_oa': None, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S147353869', 'iss...",...,"[{'id': 'https://openalex.org/S147353869', 'di...","[https://openalex.org/W618738618, https://open...","[https://openalex.org/W1994686255, https://ope...",https://api.openalex.org/works/W3164306728/ngrams,None,https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2022, 'cited_by_count': 5}]",2023-02-19T00:18:31.811719,2021-06-07,NaN
4,https://openalex.org/W2946260620,https://doi.org/10.11646/zootaxa.4608.2.1,A story of nasal horns: two new subspecies of ...,A story of nasal horns: two new subspecies of ...,0.099213,2019,2019-05-20,{'openalex': 'https://openalex.org/W2946260620...,"{'is_oa': None, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S171471881', 'iss...",...,"[{'id': 'https://openalex.org/S171471881', 'di...",[https://openalex.org/W2473892951],"[https://openalex.org/W1972796382, https://ope...",https://api.openalex.org/works/W2946260620/ngrams,"{'The': [0, 126, 138, 312], 'Lesser': [1, 22, ...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2022, 'cited_by_count': 6}, {'year':...",2023-02-19T16:21:52.369343,2019-05-29,NaN
...,...,